In [1]:
import pyodbc
import pandas as pd
from datetime import date, timedelta, datetime
import time
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#from fuxion_analytics_build_historicals.src import fuxion_tools
#%matplotlib inline
#sns.set(style='darkgrid')

## Obtención de Data del Servidor

In [2]:
OUTPUT_PATH = '/Users/carlosalbertoleonliza/devopsfuxion/fuxionbiotech/data/processed/'
SOPORTE_PATH = '/Users/carlosalbertoleonliza/devopsfuxion/fuxionbiotech/reports/'

In [3]:
periodo = 572 # 2019:468, 2020:520, 2021:572

In [4]:
# Configuración General del servidor en gcp
username = 'cleon'
password = 'carlos1304'
server = '35.229.119.253'
port = 1433
driver = '/usr/local/lib/libtdsodbc.so'
database = 'db_analitycs'

In [5]:
PeriodoTipoID = 10
start_date = '2021-01-05'
#start_date = '2021-05-18'
end_date = '2021-10-05'

In [6]:
query_ordenes1 = (
    "SELECT CodigoPais, slice_p.PeriodoID AS SemanaFuxionID, ConsumidorID, SUM(OrdenFlag) AS CantidadOrdenes, "
    "CAST(SUM(PrecioTotal / atc.ValorPromedio) AS DECIMAL(18,3)) AS VentasProductosUSD, SUM(VolumenTotal) AS VolumenPersonal "
    "FROM db_analitycs.dbo.etl_ordenes_test o "
    "INNER JOIN "
        "(SELECT PeriodoID, FechaInicio, FechaFIN "
        "FROM db_analitycs.dbo.sop_periodos p "
        "WHERE p.PeriodoTipoID = {0} "
        "AND ('{1}' <= p.FechaInicio) AND (p.FechaFin < '{2}')) slice_p "
        "ON (slice_p.FechaInicio <= o.FechaOrden) AND (o.FechaOrden < slice_p.FechaFin + 1) "
    "INNER JOIN db_analitycs.dbo.aggw_tipo_cambio atc ON slice_p.PeriodoID=atc.SemanaFuxionID AND "
        "o.CodigoMoneda=atc.CodigoMoneda "
    "WHERE OrdenStatusID IN (7, 8, 9) "
    "AND OrdenFlag != 0 "
    "GROUP BY CodigoPais, slice_p.PeriodoID, ConsumidorID "
    "ORDER BY CodigoPais, SemanaFuxionID, ConsumidorID ASC "
).format(PeriodoTipoID, start_date, end_date)

query_ordenes2 = query = (
    "SELECT c.ConsumidorID, ct.TipoConsumidor, EstadoConsumidor, Sexo, PatrocinadorID,"
    "DATEDIFF(year,FechaNacimiento,GETDATE()) AS Edad, spc.SemanaFuxionID AS SemanaFuxionIDPrimeraCompra, c.CodigoPais "
    "FROM db_analitycs.dbo.etl_consumidores AS c " 
    "INNER JOIN db_analitycs.dbo.sop_consumidores_tipo ct ON c.TipoConsumidorID=ct.TipoConsumidorID "
    "INNER JOIN db_analitycs.dbo.sop_semana_primera_compra_test spc ON c.ConsumidorID=spc.ConsumidorID"
)

In [7]:
# Conexión al servidor SQL Server
start_time = time.time()

cnx = pyodbc.connect(server=server, database=database, user=username, tds_version='7.4', 
                     password=password, port=port, driver=driver)

# Lectura de tabla
aggw_actividad_ventas = pd.read_sql(query_ordenes1, cnx)
consumidores = pd.read_sql(query_ordenes2, cnx)

# cierre de conexión
cnx.close()

end_time = time.time()
print(f'El tiempo de ejecución es: {np.round((end_time - start_time)/60,2)} minutos')

El tiempo de ejecución es: 2.0 minutos


In [8]:
print(f'Size Ordenes : {aggw_actividad_ventas.shape}')
print('')
aggw_actividad_ventas.head()
#1166035


Size Ordenes : (411139, 6)



,CodigoPais,SemanaFuxionID,ConsumidorID,CantidadOrdenes,VentasProductosUSD,VolumenPersonal
0,AR,572,425934,1,241.342,218.0
1,AR,572,587654,1,126.103,142.0
2,AR,572,631458,1,67.193,64.0
3,AR,572,632340,1,47.778,42.0
4,AR,572,697175,1,114.475,102.0


In [9]:
print(f'Size Consumidores : {consumidores.shape}')
print('')
consumidores.head()

Size Consumidores : (941960, 8)



,ConsumidorID,TipoConsumidor,EstadoConsumidor,Sexo,PatrocinadorID,Edad,SemanaFuxionIDPrimeraCompra,CodigoPais
0,1,Staff,Active,M,0,56.0,175,PE
1,5,Distribuidor,Deleted,F,8076,55.0,248,PE
2,6,Distribuidor,Deleted,M,9258,50.0,183,PE
3,8,Distribuidor,Deleted,M,133,52.0,168,PE
4,20,Distribuidor,Deleted,M,9258,61.0,167,PE


In [10]:
print('fecha: ', datetime.today().strftime("%a %b %d %H:%M:%S"))
print('fecha: ', datetime.utcnow().strftime("%a %b %d %H:%M:%S UTC %Y"))

fecha:  Fri Oct 15 15:22:17
fecha:  Fri Oct 15 20:22:17 UTC 2021


In [11]:
network_from_table = pd.read_csv(SOPORTE_PATH + 'report_network_results.csv')
network_from_table.rename(columns={'customerid':'Leader', 'periodid': 'SemanaFuxionID'}, inplace=True)
network_from_table['Nombres'] = network_from_table['Nombres'].apply(lambda x: x.upper())
network_from_table = pd.merge(consumidores[['ConsumidorID', 'Sexo', 'Edad', 'CodigoPais']], network_from_table, 
                              left_on='ConsumidorID', right_on='Leader').drop(['ConsumidorID'], axis=1)
# reorder the columns
network_from_table = network_from_table.reindex(columns=['SemanaFuxionID', 'PeriodDescription', 'Leader', 'Nombres', 'Edad', 'Sexo', 'CodigoPais'] + 
                                                network_from_table.columns[7:].tolist())
print(network_from_table.shape)

(14976, 22)


In [12]:
network_from_table.columns

Index(['SemanaFuxionID', 'PeriodDescription', 'Leader', 'Nombres', 'Edad',
       'Sexo', 'CodigoPais', 'Rango ID', 'Rango Pagado', 'Empresarios Habiles',
       'Empresarios Activos', 'Empresarios Nuevos', '# de Inactivos',
       '% de Afiliacion', '% de Mantenimiento', '# de Patrocinados',
       'Empresarios que compraron', 'Ticket Promedio de Compra',
       'Ticket Promedio de Afiliacion', 'Puntos de Afiliacion',
       'Puntos de Recompra', 'Puntos Totales de Empresarios'],
      dtype='object')

In [13]:
df = aggw_actividad_ventas.merge(consumidores[['ConsumidorID', 'TipoConsumidor', 'SemanaFuxionIDPrimeraCompra', 
                                               'PatrocinadorID']], on='ConsumidorID', how='left')

In [14]:
df['TipoVenta'] = (df['SemanaFuxionID'] - df['SemanaFuxionIDPrimeraCompra']).map(lambda x: 'Recompra' if x!=0 else "Nuevo")
#df['PatrocinadorID'] = df['PatrocinadorID'].astype(int)
df.drop(['SemanaFuxionIDPrimeraCompra'], axis=1, inplace=True)

In [15]:
df.TipoVenta.value_counts()

Recompra    320236
Nuevo        90903
Name: TipoVenta, dtype: int64

In [16]:
# Check nulls: correct the sop_semana_primera_compra, check etl_consumidores document 
print(df[(df['PatrocinadorID'].isnull())].SemanaFuxionID.value_counts())
print('Shape de la Data antes:', df.shape)
df = df[(df['PatrocinadorID'].notnull())].reset_index(drop=True)
df['PatrocinadorID'] = df['PatrocinadorID'].astype(int)
print('Shape de la Data antes:', df.shape)

608    4
577    3
605    1
610    1
Name: SemanaFuxionID, dtype: int64
Shape de la Data antes: (411139, 9)
Shape de la Data antes: (411130, 9)


In [17]:
df.head()

,CodigoPais,SemanaFuxionID,ConsumidorID,CantidadOrdenes,VentasProductosUSD,VolumenPersonal,TipoConsumidor,PatrocinadorID,TipoVenta
0,AR,572,425934,1,241.342,218.0,Distribuidor,66080,Recompra
1,AR,572,587654,1,126.103,142.0,Distribuidor,586600,Recompra
2,AR,572,631458,1,67.193,64.0,Distribuidor,586600,Recompra
3,AR,572,632340,1,47.778,42.0,Distribuidor,631458,Recompra
4,AR,572,697175,1,114.475,102.0,Distribuidor,586600,Recompra


In [18]:
print('volumen cliente promedio:',df[df['TipoConsumidor']!='Distribuidor']['VolumenPersonal'].mean())
print('volumen EF promedio:',df[df['TipoConsumidor']=='Distribuidor']['VolumenPersonal'].mean())

volumen cliente promedio: 61.40019026136
volumen EF promedio: 98.78923362475349


In [19]:
def get_distributors_tree(leaders_lst, data):
    # Calcula el arbol de distribuidores por cada enroller. Es una lista de dicts.
    # Tambien se registra en qué nivel esta su distribuidor y la cantidad de distribuidores por nivel
    
    #leaders_lst: list, lista de enrollers o líderes
    # data: dataframe, data
    
    df = data.copy()
    distributors_tree = []
    for leader in leaders_lst:
        nivel = 0
        multilevel_ef = [{'nivel': nivel, 'cantidad_EFs':1, 'downline_ef_lst': [leader]}]
        patrocinador_upd_lst = [leader]
        while patrocinador_upd_lst:
            downline_ef = df[(df['TipoConsumidor']=='Distribuidor') & (df['PatrocinadorID'].isin(patrocinador_upd_lst))]['ConsumidorID'].\
                                    drop_duplicates().to_list()
            nivel += 1
            multilevel_ef.append({'nivel': nivel , 'cantidad_EFs': len(downline_ef), 'downline_ef_lst': downline_ef})
            patrocinador_upd_lst = downline_ef
        # remove the last item because is always empty
        multilevel_ef = multilevel_ef[:-1]
        temp = [{'leader': leader, 'multilevel_ef': multilevel_ef}]
        distributors_tree.extend(temp)
    return distributors_tree

In [20]:
def compute_network_variables(networks, data):
    # computa variables de la red de distribuidores
    # networks: list of dicts - arbol de distribuidores por nivel
    # data: dataframe - data
    
    df = data.copy()
    network_df = pd.DataFrame()

    for row in range(len(networks)):
        # flatten the downline
        flatten_downline_ef = [d['downline_ef_lst'] for d in networks[row]['multilevel_ef']]
        flatten_downline_ef = [item for sublist in flatten_downline_ef for item in sublist]
        flatten_downline_ef = list(dict.fromkeys(flatten_downline_ef))
        #print('Estamos en lider {0}, y downline {1}'.format(networks[row]['leader'], flatten_downline_ef))
        # If you want to compute the volume of customers below the leader, omit this line 
        direct_customers = df[(df['TipoConsumidor']=='Cliente') & (df['PatrocinadorID']==networks[row]['leader'])]['ConsumidorID'].drop_duplicates().to_list()
        
        temp = df[~(df['ConsumidorID'].isin(direct_customers))]
        temp = temp[temp['PatrocinadorID'].isin(flatten_downline_ef)].\
                groupby(['SemanaFuxionID', 'TipoConsumidor', 'TipoVenta']).\
                agg(VolumenRed=('VolumenPersonal', 'sum'), CantidadRed=('SemanaFuxionID', 'count'))
        temp['Leader'] = networks[row]['leader']
        
        network_df = network_df.append(temp)
    network_df.reset_index(inplace=True)
    
    return network_df

In [21]:
def get_leaders(data):
    df = data.copy()
    df.columns = df.columns.str.lower()
    leaders = df['leader'].drop_duplicates().to_list()
    return leaders

In [22]:
#leaders_lst = get_leaders(network_from_table)
leaders_lst = [1142141, 1135769, 159237] # 74348
print(len(leaders_lst))

3


In [23]:
# Tiempo de ejecución de la tarea
start_time = time.time()

networks = get_distributors_tree(leaders_lst, consumidores)

end_time = time.time()
print(f'El tiempo de ejecución es: {np.round((end_time - start_time)/60,2)} minutos')

El tiempo de ejecución es: 0.03 minutos


In [24]:
#print(networks)

In [25]:
# Tiempo de ejecución de la tarea
start_time = time.time()

network_df = compute_network_variables(networks, df)

end_time = time.time()
print(f'El tiempo de ejecución es: {np.round((end_time - start_time)/60,2)} minutos')

El tiempo de ejecución es: 0.01 minutos


In [26]:
network_df[network_df['SemanaFuxionID']==574].head()
consumidores[consumidores['ConsumidorID']==1341629]
#consumidores[(consumidores['PatrocinadorID']==1142141) & (consumidores['TipoConsumidor']=='Distribuidor')].head()
#df[df['PatrocinadorID']==1166035]
#df[(df['SemanaFuxionID']==572) & (df['PatrocinadorID'].isin(leaders_lst))]
#df[df['ConsumidorID']==1205163]
#df[df['PatrocinadorID']==1142141][:20]

,ConsumidorID,TipoConsumidor,EstadoConsumidor,Sexo,PatrocinadorID,Edad,SemanaFuxionIDPrimeraCompra,CodigoPais
936522,1341629,Cliente,Active,U,1338150,NaN,610,PE


In [27]:
# merge two dataframes
network_pivot = network_df.pivot_table(index=['SemanaFuxionID', 'Leader'], columns=['TipoConsumidor', 'TipoVenta'], 
                       values=['VolumenRed', 'CantidadRed'], fill_value=0)
flatten_column_names = ['_'.join(column_name) for column_name in network_pivot.columns.values]
network_pivot.columns = flatten_column_names
network_results = pd.merge(network_from_table, network_pivot.reset_index(), on = ['SemanaFuxionID', 'Leader'])

In [28]:
#network_results.to_csv(SOPORTE_PATH + 'report_new_network_results.csv', index=False)

In [29]:
print(network_pivot.shape)
print(network_from_table.shape)

(115, 8)
(14976, 22)


In [31]:
network_results

,SemanaFuxionID,PeriodDescription,Leader,Nombres,Edad,Sexo,CodigoPais,Rango ID,Rango Pagado,Empresarios Habiles,...,Puntos de Recompra,Puntos Totales de Empresarios,CantidadRed_Cliente_Nuevo,CantidadRed_Cliente_Recompra,CantidadRed_Distribuidor_Nuevo,CantidadRed_Distribuidor_Recompra,VolumenRed_Cliente_Nuevo,VolumenRed_Cliente_Recompra,VolumenRed_Distribuidor_Nuevo,VolumenRed_Distribuidor_Recompra
0,572,01 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,110.0,...,5416.0,5630.0,7,7,2,6,342,369,214,4705
1,573,02 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,114.0,...,6020.0,6288.0,6,13,3,9,253,663,268,5104
2,574,03 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,70,Premier Leader,111.0,...,3341.0,3341.0,6,10,0,7,334,568,0,2439
3,575,04 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,70,Premier Leader,109.0,...,1591.0,1769.0,5,14,1,4,340,819,178,432
4,576,05 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,111.0,...,1047.0,1181.0,5,5,1,4,263,398,134,386
5,577,06 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,112.0,...,2691.0,2799.0,3,9,1,5,142,459,108,2090
6,578,07 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,113.0,...,2876.0,2980.0,6,8,1,6,259,465,104,2152
7,579,08 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,110.0,...,1971.0,2076.0,10,13,1,5,610,929,105,432
8,580,09 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,109.0,...,1226.0,1270.0,2,9,1,6,162,611,44,453
9,581,10 - 2021,159237,ELBERT HERRERA GUTIERREZ,43.0,M,CO,60,Leader X,110.0,...,2891.0,2999.0,2,15,1,3,142,957,108,1792
